
# Assignment 2: Decision Tree in Rust (with PyO3 & scikit-learn)

Welcome to Assignment 2, where you’ll build on your Rust knowledge and explore integrating Python and Rust for AI tasks. You will:

1. Implement a Decision Tree classifier from scratch in Rust.
2. Compare results to scikit-learn’s Decision Tree using PyO3 calls.

**Deliverables**:
- A Rust library that implements your decision tree.
- A binary (CLI) that can train/test the tree on a CSV dataset.
- This notebook (`assignment_2.ipynb`) showing how you generated data, called Rust code, and compared results.


## Usage of AI Coding Assistant

Usage of AI Coding Assistant is permitted to assist in completing the assignment under the following conditions:

1. The code suggestions provided by the AI Coding Assistant can be used to understand the Rust programming language and complete the assignment. However, it is not allowed to copy the code produced by the AI Coding Assistant (as it is) and submit it as the solution to the assignment. You should try to understand the code suggestions and write your own code.

2. The usage of any AI Coding Assistant must be acknowledged and disclosed in the submission by writing information about the model used, the prompt(s) used to generate the code, and the parts of the code written by the AI Coding Assistant.

## AI Coding Assistant Usage Disclosure

In case you used an AI Coding Assistant to help you in completing this assignment, it is required to write a disclosure about the model used, the prompts used to generate the code, and the parts of the code generated by the AI Coding Assistant.

**Please note that using AI Coding Assistant would not affect your grade in any way as long as you disclose the required information.**

[# Write your disclosure here if applicable]

--------------------

## 1. Environment Setup & Verification

In this section, you’ll verify your environment, just like in Assignment 1.

**Steps/Notes**:
- Check if Python, Rust, Cargo, and scikit-learn are installed.
- Possibly remind students to set up a new virtual environment for Python.
- Mention any required crates: e.g., `[dependencies] pyo3 = "0.23.4"`
- Make sure to use Python 3.10+ for PyO3 compatibility.

In [49]:
# Check versions
!cargo --version
!rustc --version
!python --version

print("\n" * 3)

# Installs dependencies and maturin for pyo3
# Assumes that a venv has already been created and sourced
%pip install -r requirements.txt
!maturin develop --release

print("\n" * 3)

# Check Python packages:
!pip show rustdt
!pip show scikit-learn

cargo 1.90.0 (840b83a10 2025-07-30)
rustc 1.90.0 (1159e78c4 2025-09-14)
Python 3.14.0




Obtaining file:///Users/marconms/dev/uni/Training-ML-with-Rust (from -r requirements.txt (line 1))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rustdt (pyproject.toml) ... done
  Created wheel for rustdt: filename=rustdt-0.1.0-cp314-cp314-macosx_11_0_arm64.whl size=338878 sha256=70bfd7397d5298ce20c7fc4ed1f92b01da6843c09bb400bc9d3c42799d4fe12f
  Stored in directory: /private/var/folders/hm/6gnw9yzj0vb_8pl8_wv3g_9m0000gn/T/pip-ephem-wheel-cache-zbbn3ndu/wheels/f5/40/67/22d77d93757201047a16adb9f1d4c8796dd53a5d0d1f87ad51
Successfully built rustdt
  Attempting uninstall: rustdt
    Found existing installation: rustdt 0.1.0
    Uninstalling rustdt-0.1.0:
      Successfully uninstalled rustdt-0.1.0
Note: you may need to 

# 2. Implementing a Decision Tree in Rust

# 2.1. Core requirements

Implement the following core functionalities for a Decision Tree classifier in a Rust project (not here in the notebook):
1. Tree Building: A function (e.g., fit()) that constructs the decision tree from training data.
2. Prediction: A function (e.g., predict()) that, given a vector of features, returns a predicted label or probability distribution.

# 3. Comparison with Scikit-learn

Implement the similar functions as scikit-learn DecisionTreeClassifier that use your Rust implementation using PyO3 bindings then compare it with scikit-learn performance (runtime for both training and inference) and accuracy. Write code in Python the does the following:
1. Load a dataset.
2. Run scikit-learn DecisionTreeClassifier on the dataset.
3. Run your Rust Decision Tree on the same dataset.
4. Store the results.
5. Compare the results across implementations and datasets in a table and suitable plots.
6. Write a short summary of the comparison.

Use the following dataset for comparison (check [here](https://scikit-learn.org/stable/datasets/real_world.html) for loading instructions):
- fetch_covtype
- adult (use fetch_openml('adult', version=2))
- fetch_kddcup99

In [53]:
import time
from dataclasses import dataclass
from typing import Callable, List
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_covtype, fetch_openml, fetch_kddcup99
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.tree import DecisionTreeClassifier as SkDecisionTree
from sklearn.metrics import accuracy_score
from scipy import sparse
from rustdt import RustDecisionTreeClassifier

# Configuration
config = dict(
    n_runs=10,
    max_samples=50000,
    max_depth=10,
    min_samples_split=20,
    min_samples_leaf=10,
)


@dataclass
class BenchmarkResult:
    dataset: str
    model: str
    train_time: float
    infer_time: float
    accuracy: float


def load_covtype():
    X, y = fetch_covtype(as_frame=True, return_X_y=True)
    return X, y


def load_adult():
    X, y = fetch_openml("adult", version=2, as_frame=True, return_X_y=True)
    return X, y

def load_kddcup99():
    X, y = fetch_kddcup99(as_frame=True, return_X_y=True)
    return X, y


def preprocess(X_train, X_test):
    num_sel = make_column_selector(dtype_include=["int64", "float64"])
    cat_sel = make_column_selector(dtype_exclude=["int64", "float64"])
    preproc = ColumnTransformer(
        [
            ("num", "passthrough", num_sel),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_sel),
        ]
    )
    X_train = preproc.fit_transform(X_train)
    X_test = preproc.transform(X_test)
    if sparse.issparse(X_train):
        X_train, X_test = X_train.toarray(), X_test.toarray()
    return X_train, X_test


def run_benchmark(
    name: str,
    loader: Callable[[], tuple],
    n_runs: int = 1,
    max_samples: int | None = None,
    max_depth: int = 10,
    min_samples_split: int = 20,
    min_samples_leaf: int = 10,
) -> List[BenchmarkResult]:
    X, y = loader()
    if max_samples and len(y) > max_samples:
        X = X.sample(n=max_samples, random_state=0)
        y = y.loc[X.index] if hasattr(y, "loc") else y[:max_samples]

    results = []

    for _ in range(n_runs):
        X_train, X_test, y_train_raw, y_test_raw = train_test_split(
            X, y, test_size=0.25, random_state=0,
            # stratify=y <- kddcup99 is not suitable for stratification without filtering
        )
        y_train = LabelEncoder().fit_transform(y_train_raw)
        y_test = LabelEncoder().fit(y_train_raw).transform(y_test_raw)
        X_train, X_test = preprocess(X_train, X_test)

        sk = SkDecisionTree(
            criterion="gini",
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=0,
        )
        t0 = time.perf_counter()
        sk.fit(X_train, y_train)
        t1 = time.perf_counter()
        y_pred = sk.predict(X_test)
        t2 = time.perf_counter()
        results.append(
            BenchmarkResult(
                name, "sklearn", t1 - t0, t2 - t1, accuracy_score(y_test, y_pred)
            )
        )

        rs = RustDecisionTreeClassifier(
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
        )
        t0 = time.perf_counter()
        rs.fit(X_train, y_train)
        t1 = time.perf_counter()
        y_pred = rs.predict(X_test)
        t2 = time.perf_counter()
        results.append(
            BenchmarkResult(
                name, "rust", t1 - t0, t2 - t1, accuracy_score(y_test, y_pred)
            )
        )
    return results


def main():

    # Print config
    print("Benchmark configuration:")
    for k, v in config.items():
        print(f"  {k}: {v}")
    print()

    all_results = []
    for name, loader in [
        ("covtype", load_covtype),
        ("adult", load_adult),
        ('kddcup99', load_kddcup99)
    ]:
        print(f"Running {name} ...")
        all_results += run_benchmark(name, loader, **config)

    df = pd.DataFrame(r.__dict__ for r in all_results)
    summary = (
        df.groupby(["dataset", "model"]).mean(numeric_only=True).reset_index().round(4)
    )

    print("\nAverage results over", config["n_runs"], "runs:\n")
    print(summary.to_string(index=False))

    for dataset in summary["dataset"].unique():
        dset = summary[summary["dataset"] == dataset]
        sk, rs = (
            dset[dset["model"] == "sklearn"].iloc[0],
            dset[dset["model"] == "rust"].iloc[0],
        )
        print(f"\nDataset: {dataset}")
        print(
            f"  rust   : acc={rs.accuracy:.4f}, train={rs.train_time:.4f}s, infer={rs.infer_time:.4f}s"
        )
        print(
            f"  sklearn: acc={sk.accuracy:.4f}, train={sk.train_time:.4f}s, infer={sk.infer_time:.4f}s"
        )


if __name__ == "__main__":
    main()


Benchmark configuration:
  n_runs: 10
  max_samples: 50000
  max_depth: 10
  min_samples_split: 20
  min_samples_leaf: 10

Running covtype ...
Running adult ...
Running kddcup99 ...

Average results over 10 runs:

 dataset   model  train_time  infer_time  accuracy
   adult    rust      0.0588      0.0006    0.8618
   adult sklearn      0.1223      0.0011    0.8624
 covtype    rust      0.0418      0.0005    0.7574
 covtype sklearn      0.1661      0.0011    0.7570
kddcup99    rust      3.5263      0.2657    0.9974
kddcup99 sklearn      4.2270      0.0467    0.9974

Dataset: adult
  rust   : acc=0.8618, train=0.0588s, infer=0.0006s
  sklearn: acc=0.8624, train=0.1223s, infer=0.0011s

Dataset: covtype
  rust   : acc=0.7574, train=0.0418s, infer=0.0005s
  sklearn: acc=0.7570, train=0.1661s, infer=0.0011s

Dataset: kddcup99
  rust   : acc=0.9974, train=3.5263s, infer=0.2657s
  sklearn: acc=0.9974, train=4.2270s, infer=0.0467s


##### Instructions
Before running, please make sure to install the python packages specified in the `requirements.txt` file and compile the project with `!maturin develop --release`. For your convenience, the cell in section 1 of this notebook has been updated with the necessary commands. The commands assume that a virtual environment has already been created and sourced before running the commands.

##### Description of the implementation
The Rust class `RustDecisionTreeClassifier` exposes the two methods to Python:
- fit(x, y): trains the model with the provided dataset and labels
- predict(x): predicts the label for the provided datapoints

The implementation wraps a parallel Rust decision tree using a thin Python interface built with PyO3. Data is passed as NumPy arrays for zero-copy conversion into Rust’s ndarray types. Heavy computation runs fully in Rust, parallelized with Rayon, while Python's Global Interpeter Lock is released as soon as possible to allow other Python threads to run if necesary.

We relied heavility on scikit-learn's own `DecisionTreeClassifier` for our implementation, due to the limited time given for the task and the wish to achieve an efficient implementation of the classifier.

##### Interpretation of the benchmark results
- The Rust classifier matches scikit-learn’s accuracy almost exactly and consistently.
- Training and inference are generally about ~2–4× faster in out Rust implementation, with the exception of inference in the kddcup99 dataset. [TODO: insert motivation]
- The implementation and the achieved performance gains clearly demonstrate the advantages of choosing Rust for resource-intensive ML application, both in training and inference. Such a big differnce could save a lot of time and resources for the training of bigger AI models.

##### Resources utilized
- Pyo3's docs: https://pyo3.rs/v0.27.1/
- scikit-learn's DecisionTreeClassifier class: https://github.com/scikit-learn/scikit-learn/blob/4e2f1b7094d27ddca17bee1eee61af2ab20a7d23/sklearn/tree/_tree.pxd

#### Testing environment
OS: macOS 26.0.1 (25A362)\
cargo 1.90.0 (840b83a10 2025-07-30)\
rustc 1.90.0 (1159e78c4 2025-09-14)\
Python 3.14.0

# 4. Submission

**What to submit**: (all in one zip file named: `EffAIRust_assignment_2_Team_[TeamID].zip`):
1. Your completed Rust project (with `Cargo.toml`, `src/`, tests, etc.).
2. This notebook (`assignment_2.ipynb`), so we can see your test results.

Submit the assignment as a ZIP file to Moodle.